### Lets build chatbots using langchain

In [1]:
from langchain_groq import ChatGroq
from langchain.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
model = ChatGroq(model="llama-3.1-8b-instant")

In [3]:
# lets invoke the model
model.invoke([HumanMessage(content="Hi, my name is srini!")])

AIMessage(content="Nice to meet you, Srini. What brings you here today? Do you have any questions or topics you'd like to discuss?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 43, 'total_tokens': 71, 'completion_time': 0.03664214, 'completion_tokens_details': None, 'prompt_time': 0.00211933, 'prompt_tokens_details': None, 'queue_time': 0.005667805, 'total_time': 0.03876147}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9ca2574dca', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--21cb0172-2eb5-4e83-b588-99e23c1d3c83-0', usage_metadata={'input_tokens': 43, 'output_tokens': 28, 'total_tokens': 71})

In [4]:
model.invoke([HumanMessage(content="What is my name?")])

AIMessage(content="I don't have any information about your name as our conversation has just started. I'm a large language model, I don't have the ability to retain information about individual users or recall previous conversations. Each time you interact with me, it's a new conversation. If you'd like to share your name with me, I'd be happy to chat with you!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 40, 'total_tokens': 114, 'completion_time': 0.093669409, 'completion_tokens_details': None, 'prompt_time': 0.001827275, 'prompt_tokens_details': None, 'queue_time': 0.005696141, 'total_time': 0.095496684}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4dea31877a', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--0ecd352f-69cd-43cf-97ea-25d5d0e908bf-0', usage_metadata={'input_tokens': 40, 'output_tokens': 74, 'total_tokens': 114})

In [5]:
# lets invoke the model
model.invoke(
    [
        HumanMessage(content="Hi, my name is srini!"),
        AIMessage(content="Nice to meet you, Srini! I'm happy to chat with you. Is there something on your mind that you'd like to talk about, or do you just want to say hello?"),
        HumanMessage(content="What is my name?")
    ]
)

AIMessage(content='Your name is Srini.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 97, 'total_tokens': 104, 'completion_time': 0.009115773, 'completion_tokens_details': None, 'prompt_time': 0.005578157, 'prompt_tokens_details': None, 'queue_time': 0.005326363, 'total_time': 0.01469393}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9ca2574dca', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--bc9f76a7-c864-4aad-a778-e5cb7c1e55e4-0', usage_metadata={'input_tokens': 97, 'output_tokens': 7, 'total_tokens': 104})

As we see here:
- If we do not provide the context. LLM will not have any information
- If we provide the context, it can easily know and answer based on context

This is called illusion of memory

### Message History

We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# history store
store = {}

In [7]:
# create a function to get the chat history based on session id

def get_chat_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [ ]:
# setup config
config = {'configurable': {'session_id': 'chat_1'}}

# setup model with the session history
with_chat_history = RunnableWithMessageHistory(model, get_session_history=get_chat_history)

In [19]:
# interact with model with session history
response = with_chat_history.invoke(
    [
        HumanMessage(content="Hi, my name is srini! I am Senior GenAI Engineer!"),
    ],
    config=config
)

In [20]:
response.content

"It seems like you've repeated your introduction, Srini. Don't worry, I've got it noted! You're a Senior GenAI Engineer. What would you like to talk about? Would you like to discuss a specific AI-related topic, or do you have a question for me?"

In [22]:
# change the config - new session id
config1 = {'configurable': {'session_id': 'chat_2'}}

# interact with model with session history
response = with_chat_history.invoke(
    [
        HumanMessage(content="Hi, What is my name?"),
    ],
    config=config1
)

response.content

"You still haven't told me your name. I'm happy to chat with you, but I don't know what to call you. If you'd like to share your name, I can try to recall it and use it in our conversation."

In [23]:
# interact with model with session history - with the first session
response = with_chat_history.invoke(
    [
        HumanMessage(content="Hi, What is my name?"),
    ],
    config=config
)

response.content

'Your name is Srini.'